In [1]:
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS  # Importar CORS
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import torch
from threading import Thread

In [2]:
# Crear la app de Flask
app = Flask(__name__)

# Habilitar CORS para todas las rutas
CORS(app)  # Esto habilita CORS para todos los orí­genes

# Cargar el modelo
model_name = "facebook/nllb-200-distilled-600M"     # Versión compacta
# model_name = "facebook/nllb-200-3.3B"            # Mejor calidad
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# For English to Spanish
src_lang = "eng_Latn"
tgt_lang = "spa_Latn"

# Cargar contenido HTML desde un archivo
def load_html(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return f.read()

# Ruta para servir el HTML
@app.route("/")
def home():
    html_content = load_html("templates/page.html")
    return render_template_string(html_content)

# Ruta para el cambio de idiomas
@app.route("/switch", methods=["POST"])
def switch_lang():
    data = request.json
    global src_lang
    global tgt_lang
    if  data["source"] == "Inglés" and data["target"] == "Español":
        src_lang = "spa_Latn"
        tgt_lang = "eng_Latn"
        return jsonify({"source":"Español", "target":"Inglés"})
    else:
        src_lang = "eng_Latn"
        tgt_lang = "spa_Latn"
        return jsonify({"source":"Inglés", "target":"Español"})
    
# Ruta para la traducción
@app.route("/translate", methods=["POST"])
def translate():
    translator = pipeline("translation", model=model, tokenizer=tokenizer, src_lang=src_lang, tgt_lang=tgt_lang, max_length = 400)
    data = request.json
    output = translator(data["text"])
    return jsonify({"translate": output[0]["translation_text"]})

In [ ]:
# Ejecutar Flask en un hilo
def run_flask():
    app.run(host="0.0.0.0", port=5555)

# Crear un hilo para ejecutar Flask
thread = Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5555
 * Running on http://172.82.3.25:5555
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2025 19:33:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 19:33:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Mar/2025 19:33:35] "OPTIONS /translate HTTP/1.1" 200 -
Device set to use cuda:0
127.0.0.1 - - [15/Mar/2025 19:33:39] "POST /translate HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 19:33:43] "OPTIONS /switch HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 19:33:43] "POST /switch HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 19:33:48] "OPTIONS /translate HTTP/1.1" 200 -
Device set to use cuda:0
127.0.0.1 - - [15/Mar/2025 19:33:48] "POST /translate HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 19:36:30] "GET / HTTP/1.1" 200 -
